# 主要工具包

- `pip install opencv-python -i https://pypi.tuna.tsinghua.edu.cn/simple`
- `pip install PyPDF2 -i https://pypi.tuna.tsinghua.edu.cn/simple`

# 结构分析

In [ ]:
from PyPDF2 import PdfReader, PdfWriter
from PIL import Image
import numpy as np
import cv2

path=r"D:\pdf\第一册.pdf"

reader = PdfReader(open(path, 'rb'))
print(len(reader.pages))

"""
注意PyPDF2中页码从0开始
"""
# print(reader.pages[4])
blank=reader.pages[19]
xobject_blank=blank['/Resources']['/XObject'].get_object()
print(blank)
print(xobject_blank)
count = 0
for image_file_object in blank.images:
    print(image_file_object)
    # 二进制数据流转np.ndarray [np.uint8: 8位像素]
    img = cv2.imdecode(np.frombuffer(image_file_object.data, np.uint8), cv2.IMREAD_COLOR)
    # 将bgr转为rbg
    rgb_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    # np.ndarray转IMAGE
    a = Image.fromarray(rgb_img)
    # a.show()
    print(np.sum(a))
    print(np.mean(np.array(a)))
    np.reshape(a, (-1,1))
    u, count_unique = np.unique(a, return_counts = True)
    print(count_unique.size)
    print(np.std(img))

    # 显示图片

    # byte_array = bytearray(image_file_object.data)
    # numpy_array = np.frombuffer(byte_array, dtype=np.uint8)
    # # img=np.array(image_file_object.data)
    # # img=Image.frombytes('L',[blank.mediabox.width.to_integral_value(), blank.mediabox.height.to_integral_value()], image_file_object.data, "raw")
    # img.show()
    # print(np.mean(np.array(img)))
    # print(np.mean(numpy_array))

In [1]:
from PyPDF2 import PdfReader, PdfWriter

path = r"D:\pdf\第七册.pdf"
path_output = r"D:\pdf\第七册output.pdf"

reader = PdfReader(open(path, 'rb'))
writer = PdfWriter()
pages = len(reader.pages)

for i in range(pages):
    page = reader.pages[i]
    if "/XObject" in page["/Resources"].keys() or "/Font" in page["/Resources"].keys():
        writer.add_page(page)
    
# writer.write(open(path_output, 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: '第七册.pdf'

In [ ]:
from PyPDF2 import PdfReader, PdfFileWriter

path=r"第七册.pdf"

local = '/Users/kkstar/Downloads/'

reader = PdfReader(open(path, 'rb'))
num_pages = len(reader.pages)

# 遍历每一页
for page_num in range(num_pages):
    # 获取当前页对象
    page_obj = reader.pages[page_num]
    # 获取当前页中的所有对象
    for image_file_object in page_obj.images:
    # 遍历每个对象
        for obj_name in page_objs:
            # 判断对象是否为图片
            if page_objs[obj_name]['/Subtype'] == '/Image':
                # 获取图片对象
                img_obj = page_objs[obj_name]
                # 获取图片数据
                img_data = img_obj.get_data()
                # 将图片数据保存为文件
                with open(local+obj_name + '.jpg', 'wb') as img_file:
                    img_file.write(img_data)

"""
注意PyPDF2中页码从0开始
"""
print(reader.pages[4])
print(reader.pages[5])
print(reader.pages[6])
print(reader.pages[7])
blank=reader.pages[5]
xobject_blank=blank['/Resources']['/XObject'].get_object()
img_data = xobject_blank.get_data()
print(xobject_blank)

# 清洗文档-删除空白页-扫描生成

扫描文档生成的空白页图像主要的数据特征为像素少且较均匀；但由于双面打印文档容易引起反面内容的渗透，导致一些空白页图像与一些内容很少的非空白页在统计特征上相似，所以最好的识别方式还是通过OCR识别出不包含文字且统计特性也相符（像素少且较均匀）的页面。

空白页图像统计特征：

- 低熵
  - 图像的一维熵表示图像中灰度分布的聚集特征所包含的信息量；反映了图像中纹理的紊乱度或复杂程度。熵值越大，说明纹理越复杂；熵值越小，说明纹理越平滑。
  - 图像的一维熵不能反映图像灰度分布的空间特征，为了表征这种空间特征，可以在一维熵的基础上引入能够反映灰度分布空间特征的特征量来组成图像的二维熵。
- 高均值？
  - 灰度均值是对区域内亮度的一个度量，可以用来反应图像的明暗程度。
- 低方差
  - 方差就是数据的分散程度（偏离均值）。
- 低对比度
  - 反映了图像的清晰度和纹理沟纹深浅的程度。纹理沟纹越深，其对比度越大，视觉效果越清晰；反之，对比度小，则沟纹浅，效果模糊。


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from PyPDF2 import PdfReader
import numpy as np
import cv2
import plotly.express as px

filename=r'D:\pdf\第一册.pdf'
reader=PdfReader(open(filename,'rb'))
num_pages = len(reader.pages)
data = []
for page_num in range(num_pages):
    page_obj = reader.pages[page_num]
    for image_file_object in page_obj.images:
        # 提取图像并转换为灰度图
        img_gray = cv2.imdecode(np.frombuffer(image_file_object.data, np.uint8), cv2.IMREAD_COLOR)
        # 计算图像均值
        img_mean = np.mean(img)
        # 计算图像方差
        img_std=np.std(img)
        # 计算图像直方图
        hist, _ = np.histogram(img_gray, bins=256, range=[0, 256])
        # 计算直方图概率分布
        prob = hist / (img_gray.shape[0] * img_gray.shape[1])
        # 计算图像信息熵
        img_entropy = -np.sum(prob * np.log2(prob + np.finfo(float).eps))
        data.append(img_mean,img_std,img_entropy)
# 创建一个DataFrame来存储统计特征
df = pd.DataFrame(data, columns=["img_mean","img_std","img_entropy"])
print(df.head(10))

## 低方差筛选

采用较低门限，确保删除页面的准确性。

In [ ]:
from PyPDF2 import PdfReader, PdfWriter
import numpy as np
import cv2

# 检测空白页
def not_blank_page (filename, page_num) :
    with open(filename, 'rb') as f:
        pdf_reader = PdfReader(f)
        page_obj = pdf_reader.pages[page_num]
        for image_file_object in page_obj.images:
            img = cv2.imdecode(np.frombuffer(image_file_object.data, np.uint8), cv2.IMREAD_COLOR)
            std_dev=np.std(img)
            # 门限值可通过聚类分析而得
            if std_dev > 3:
                return True
        return False

if __name__ == '__main__':
    filename='D:\pdf\第一册.pdf'
    reader=PdfReader(open(filename,'rb'))
    writer=PdfWriter()
    num_pages = len(reader.pages)
    for page_num in range (num_pages):
        if not_blank_page(filename, page_num):
            pageObj = reader.pages[page_num]
            writer.add_page(pageObj)
            # print(f'空白页: {pdf_file} 第{page_num+1}页')
    writer.write(open(f'D:\pdf\out1_第一册.pdf', 'wb'))  

## 离群值识别
### 孤立森林

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from PyPDF2 import PdfReader
import numpy as np
import cv2
import plotly.express as px

filename=r'D:\pdf\第一册.pdf'
reader=PdfReader(open(filename,'rb'))
num_pages = len(reader.pages)
data = []
for page_num in range(num_pages):
    page_obj = reader.pages[page_num]
    for image_file_object in page_obj.images:
        img = cv2.imdecode(np.frombuffer(image_file_object.data, np.uint8), cv2.IMREAD_COLOR)
        std_dev=np.std(img)
        data.append(std_dev)
# 创建一个DataFrame来存储统计指标
df = pd.DataFrame(data, columns=["img_std"])
print(df.head(10))

# 通过去除平均值并缩放至单位方差来标准化特征。
# standardizer = StandardScaler()
# analysisstand = standardizer.fit_transform(results.iloc[:, 1:])

# 构建100颗树孤立森林模型
model=IsolationForest(n_estimators=100, 
  max_samples='auto', contamination=float(0.1), 
  max_features=1.0, random_state=12345)
# 训练模型
model.fit(df[['img_std']])
# 模型是否异常的判断，-1为异常，1为正常
df['anomaly'] = \
  model.predict(df[['img_std']])
# 异常评分
df['scores'] = \
  model.decision_function(df[['img_std']])
df.anomaly.value_counts()

print(df.head(10))
# 输出异常值
# inlier, outlier = \
#   df.loc[df.anomaly==1],\
#   df.loc[df.anomaly==-1]
  
fig = px.histogram(df,x='scores',color='anomaly') 
fig.show()

     img_std
0  38.692608
1  28.555372
2  27.600217
3  22.090993
4  25.546189
5   2.270433
6  25.667890
7   2.303401
8  27.449383
9  22.921607
     img_std  anomaly    scores
0  38.692608        1  0.124608
1  28.555372        1  0.074653
2  27.600217        1  0.070108
3  22.090993        1  0.011705
4  25.546189        1  0.036473
5   2.270433        1  0.076393
6  25.667890        1  0.035305
7   2.303401        1  0.088691
8  27.449383        1  0.071011
9  22.921607        1  0.042402
